In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.dates as mdates
import datetime as dt
import matplotlib.pyplot as plt
from scipy.stats import pearsonr as pr
from scipy import signal
from scipy.signal import correlate


In [2]:
import os
import sys

In [3]:
indice = pd.read_csv("C:/Users/Anne Ultelie Poinçon/Desktop/PracticePython/inputs/hm_ARVI.csv")
meteo = pd.read_csv("C:/Users/Anne Ultelie Poinçon/Desktop/PracticePython/inputs/hm_tempPrecip.csv")
shortwave= pd.read_csv("C:/Users/Anne Ultelie Poinçon/Desktop/PracticePython/inputs/shortwave.csv")

In [4]:
sys.path.append(os.path.join(os.path.dirname('__file__'), 'utils'))

In [5]:
months = {
    1: {"name":"Jan", "end":31},
    2: {"name":"Feb", "end":28},
    3: {"name":"Mar", "end":31},
    4: {"name":"Apr", "end":30},
    5: {"name":"May", "end":31},
    6: {"name":"June", "end":30},
    7: {"name":"Jul", "end":31},
    8: {"name":"Aug", "end":31},
    9: {"name":"Sep", "end":30},
    10: {"name":"Oct", "end":31},
    11: {"name":"Nov", "end":30},
    12: {"name":"Dec", "end":31},
}
from datetime import datetime

def str_to_float(num_str):
    try:
        nums = num_str.split(",")
        if len(nums) >= 2:
            val = float("".join(nums))
        else:
            val = float(num_str)
        return val
    except Exception as e:
        return "No"
        print(e)
        
def formatStarter(day):
    if day < 10:
        return "0" + str(day)
    return str(day)

def set_end_intevals(starter, endPos, date, month_number):
    
    if date.day < endPos and date.month == month_number:
        #return formatStarter(starter) + " - " + str(endPos) + " " + months[month_number]["name"] + " " + str(date.year)
        return datetime(date.year, date.month, endPos).date()
    elif date.day > endPos and date.month == month_number:
         return datetime(date.year, date.month, endPos+1).date()
        
#return formatStarter(endPos+1) + " - " + str(months[month_number]["end"]) + " "+ months[month_number]["name"] + " " + str(date.year)
    

def rename_intervals(starter, endPos, date, month_number):
    if date.day == endPos and date.month == month_number:
        return formatStarter(starter) + " - " + str(endPos) + " " + months[month_number]["name"] + " " + str(date.year)
    elif date.day > endPos and date.month == month_number:
        return formatStarter(endPos+1) + " - " + str(months[month_number]["end"]) + " "+ months[month_number]["name"] + " " + str(date.year)

In [6]:
shortwave.head()

,system:time_start,surface_solar_radiation_downwards
0,"Jan 1, 2018","5,221,649.833"
1,"Jan 2, 2018","4,335,803.333"
2,"Jan 3, 2018","2,117,488.5"
3,"Jan 4, 2018","1,309,567.667"
4,"Jan 5, 2018","2,322,941.5"


In [7]:
shortwave.tail()

,system:time_start,surface_solar_radiation_downwards
905,"Jun 24, 2020","8,155,515.417"
906,"Jun 25, 2020","7,811,999.333"
907,"Jun 26, 2020","7,865,486.667"
908,"Jun 27, 2020","7,910,742.083"
909,"Jun 28, 2020","7,899,381"


In [8]:
indice.head()

,system:time_start,ARVI
0,8-Jan-18,1.317
1,9-Jan-18,0.954
2,8-Feb-18,0.995
3,9-Feb-18,0.859
4,8-Mar-18,0.941


In [9]:
meteo.head()

,system:time_start,mean_2m_air_temperature,total_precipitation
0,"Jan 8, 2018",25.562,NaN
1,"Jan 8, 2018",NaN,1.96
2,"Jan 9, 2018",25.885,NaN
3,"Jan 9, 2018",NaN,0.42
4,"Feb 8, 2018",25.710,NaN


In [10]:
indice.tail()

,system:time_start,ARVI
55,9-Apr-20,0.627
56,8-May-20,0.507
57,9-May-20,0.593
58,8-Jun-20,0.572
59,9-Jun-20,0.613


In [11]:
meteo.tail()

,system:time_start,mean_2m_air_temperature,total_precipitation
117,"Jun 8, 2020",NaN,0.636
118,"Jun 9, 2020",27.808,NaN
119,"Jun 9, 2020",NaN,0.506
120,"Jul 8, 2020",28.356,NaN
121,"Jul 8, 2020",NaN,0.246


In [12]:
meteo.columns = ['date','temperature', 'precipitation']

In [13]:
indice.columns = ['date','ARVI']
#indice['date'] = [dt.datetime.strptime(d,'%B %d, %Y').date() for d in indice['date']]

In [14]:
shortwave.columns=['date','shortwaves']

In [15]:
shortwave_df= shortwave.copy()

In [16]:
indice_df= indice.copy()

In [17]:
shortwave_df= shortwave_df.set_index('date')

In [18]:
indice_df=indice_df.set_index('date')

In [19]:
shortwave.head()

,date,shortwaves
0,"Jan 1, 2018","5,221,649.833"
1,"Jan 2, 2018","4,335,803.333"
2,"Jan 3, 2018","2,117,488.5"
3,"Jan 4, 2018","1,309,567.667"
4,"Jan 5, 2018","2,322,941.5"


In [20]:
indice.head()

,date,ARVI
0,8-Jan-18,1.317
1,9-Jan-18,0.954
2,8-Feb-18,0.995
3,9-Feb-18,0.859
4,8-Mar-18,0.941


In [21]:
indice.shape

(60, 2)

In [22]:
meteo.head(2)

,date,temperature,precipitation
0,"Jan 8, 2018",25.562,NaN
1,"Jan 8, 2018",NaN,1.96


In [23]:
meteo.shape

(122, 3)

In [24]:
meteo_temp= meteo[["date", "temperature"]]

In [25]:
meteo_temp.shape

(122, 2)

In [26]:
meteo_temp=meteo_temp.dropna()

In [27]:
meteo_temp.shape

(61, 2)

In [28]:
meteo_precip=meteo[["date", "precipitation"]]

In [29]:
meteo_precip.shape

(122, 2)

In [30]:
meteo_precip.head()

,date,precipitation
0,"Jan 8, 2018",NaN
1,"Jan 8, 2018",1.96
2,"Jan 9, 2018",NaN
3,"Jan 9, 2018",0.42
4,"Feb 8, 2018",NaN


In [31]:
meteo_precip = meteo_precip.dropna()

In [32]:
meteo_precip.shape

(61, 2)

In [33]:
shortwave= shortwave.dropna()

In [34]:
shortwave.shape

(910, 2)

In [35]:
date1= list(meteo_temp['date'])

In [36]:
date2= list(meteo_precip['date'])

In [37]:
for i in range(len(date1)):
    assert date1[i]==date2[i]

In [38]:
df=meteo_temp.merge(meteo_precip)

In [39]:
df['date'] = [dt.datetime.strptime(d,'%d-%b-%y').date().strftime("%b %d, %Y") for d in df['date']]
meteo_temp['date'] = [dt.datetime.strptime(d,'%d-%b-%y').date().strftime("%b %d, %Y") for d in meteo_temp['date']]
meteo_precip['date'] = [dt.datetime.strptime(d,'%d-%b-%y').date().strftime("%b %d, %Y") for d in meteo_precip['date']]
indice['date'] = [dt.datetime.strptime(d,'%d-%b-%y').date().strftime("%b %d, %Y") for d in indice['date']]
shortwave['date'] = [dt.datetime.strptime(d,'%d-%b-%y').date().strftime("%b %d, %Y") for d in shortwave['date']]

ValueError: time data '8-Jan-18' does not match format '%b %d, %Y'

In [ ]:
shortwave.date = shortwave['date']

In [ ]:
shortwave = shortwave.sort_values(by="date")

In [ ]:
shortwave_dates_col = [set_end_intevals(1, 16, d, d.month)  for d in list(shortwave.date)]
shortwave["date_intervals"] = shortwave_dates_col
shortwave.head()

In [ ]:
shortwave_years = [d.year for d in  list(shortwave.date)]
shortwave["years"] =shortwave_years

In [ ]:
shortwave.head()

In [ ]:
shortwave.shape

In [ ]:
shortwave.shortwaves=[str_to_float(num) for num in list(shortwave.shortwaves)]

In [ ]:
shortwave.head()

In [ ]:
grouped_shortwave = shortwave.groupby(["date_intervals"])[["shortwaves"]].mean()
grouped_shortwave = grouped_shortwave.reset_index()

In [ ]:
grouped_shortwave.shape

In [ ]:
intervals = list(grouped_shortwave["date_intervals"])
new_inter = []

for d in intervals:
   new_inter.append(rename_intervals(1, 16, d, d.month))

In [ ]:
sns.set(rc={'figure.figsize':(20,6)})
fig, ax = plt.subplots(sharey=True, sharex=True)
x_pos = np.arange(len(new_inter))
_ = ax.plot(list(grouped_shortwave["shortwaves"]))
ax.set_xticks(x_pos)
rects = ax.set_xticklabels(x_pos)
_ = ax.set_xticklabels(new_inter,  rotation = 80)
plt.savefig('shortwave_halfmonth.png',dpi=250,bbox_inches = 'tight')

In [ ]:
df=df.set_index('date')

In [ ]:
df.shape

In [ ]:
df.head(2)

In [ ]:
plt.style.use("tableau-colorblind10")
print(plt.style.available)

In [ ]:
axe=df.plot()

In [ ]:
meteo.describe()

In [ ]:
meteo.info()

In [ ]:
# ------------- Plot the indexes ----------
#Initialize plot and parameters
sns.set(rc={'figure.figsize':(16,6)})
#sns.set_context('paper')

#Plot colors
prec_color = 'cornflowerblue'
#temp_color = 'fuchsia'
short_color = 'red'

In [ ]:
#Create the figure with a date x-axis
plt.style.use("tableau-colorblind10")
fig,ax = plt.subplots(2,1, sharex=True)


plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y_%b'))
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval = 60))
#ax.set_xlim(dt.date(2017,12,1),dt.date(2020,2,1))
ax[1].set_xlim(df.index.min(),df.index.max())



#Plot precipitation
ax2 = ax[1].twinx()


p_prec = ax2.bar(meteo_precip['date'],meteo_precip['precipitation'], width = 10, alpha = .75, color = prec_color,label = 'precipation(mm)')
ax2.set_ylim(0,5)
ax2.grid(False)
ax2.set_ylabel('Monthly total precipitation (mm)',fontsize = 12)
ax2.spines['right'].set_color(prec_color)
ax2.tick_params(axis='y', colors = prec_color)
ax2.yaxis.label.set_color(prec_color)

#Plot temperature
#p_temp = ax[1].plot(meteo_temp['date'],meteo_temp['temperature'],'-',color =temp_color,label = 'T (°C)')
#ax2.spines['left'].set_color(temp_color)
#ax[1].tick_params(axis='y',colors = temp_color)
#ax[1].yaxis.label.set_color(temp_color)
#ax[1].set_ylabel('Monthly mean temperature (°C)', fontsize = 12)
#ax[1].yaxis.tick_right()

#Plot shortwave
p_short = ax[1].plot(grouped_shortwave['date_intervals'],grouped_shortwave['shortwaves'],'-',color =short_color,label = 'j/m2 (°C)')
ax2.spines['left'].set_color(short_color)
ax[1].tick_params(axis='y',colors = short_color)
ax[1].yaxis.label.set_color(short_color)
ax[1].set_ylabel('Monthly mean shortwave (°C)', fontsize = 12)
#ax[1].yaxis.tick_right()



#plot ARVI
ax[0].set_ylabel('NDVI, NDWI, EVI and ARVI',fontsize=12)
ax[0].set_title('ARVI, temperature and precipitations in Haiti for the period 2018-2020/half month time step',fontsize = 14)

ax[0].plot(indice['date'],indice['ARVI'],linewidth = 2, color = 'orange', label ='ARVI')
#ax.set_ylabel('ARVI')
ax[0].legend(loc = 'upper right')


ax[1].set_xlabel('Date',fontsize = 12)
plt.savefig('short_prec_halfmonth.png',dpi=250,bbox_inches = 'tight')

In [ ]:
#crosscorellation
#fig1,ax1=plt.subplots(2, 4, figsize=(16,6), sharex= False, sharey= True )
#fig1.tight_layout(pad=3.0)
#fig1.suptitle("Correlation: Precipitation/Temperature versus indices", fontsize = 14, y=1.05)

#precipitation_list = list(meteo_precip["precipitation"])
temperature_list = list(meteo_temp["temperature"])
arvi_list = list(indice["ARVI"])

def lag_finder(temperature_list, arvi_list, sr):
    n = len(temperature_list)

    corr = signal.correlate(arvi_list, temperature_list, mode='same') / np.sqrt(signal.correlate(temperature_list, temperature_list, mode='same')[int(n/2)] * signal.correlate(arvi_list, arvi_list, mode='same')[int(n/2)])

    delay_arr = np.linspace(-0.5*n/sr, 0.5*n/sr, n)
    delay = delay_arr[np.argmax(corr)]
    print('arvi_list is ' + str(delay) + ' behind temperature_list')

    plt.figure()
    plt.plot(delay_arr, corr)
    plt.title('Lag: ' + str(np.round(delay, 3)) + 'month'
    plt.xlabel('Lag')
    plt.ylabel('Correlation coefficient')
    plt.show()
    
# Sine sample with some noise and copy to y1 and y2 with a 1-second lag
sr = 10
y = np.linspace(0, 2*np.pi, sr)
y = np.tile(np.sin(y), 5)
y += np.random.normal(0, 5, y.shape)
temperature_list = y[sr:4*sr]
arvi_list = y[:3*sr]

lag_finder(temperature_list, arvi_list, sr)
plt.savefig('lagtemp_halfmonth.png',dpi=250,bbox_inches = 'tight')

In [ ]:
precipitation_list = list(meteo_precip["precipitation"])
#temperature_list = list(meteo_temp["temperature"])
arvi_list = list(indice["ARVI"])

def lag_finder(precipitation_list, arvi_list, sr):
    n = len(precipitation_list)

    corr = signal.correlate(arvi_list, precipitation_list, mode='same') / np.sqrt(signal.correlate(precipitation_list, precipitation_list, mode='same')[int(n/2)] * signal.correlate(arvi_list, arvi_list, mode='same')[int(n/2)])

    delay_arr = np.linspace(-0.5*n/sr, 0.5*n/sr, n)
    delay = delay_arr[np.argmax(corr)]
    print('arvi_list is ' + str(delay) + ' behind precipitation_list')

    plt.figure()
    plt.plot(delay_arr, corr)
    plt.title('Lag: ' + str(np.round(delay, 3)) + 'month')
    plt.xlabel('Lag')
    plt.ylabel('Correlation coefficient')
    plt.show()
    
# Sine sample with some noise and copy to y1 and y2 with a 1-second lag
sr = 10
y = np.linspace(0, 2*np.pi, sr)
y = np.tile(np.sin(y), 5)
y += np.random.normal(0, 5, y.shape)
precipitation_list = y[sr:4*sr]
arvi_list = y[:3*sr]

lag_finder(precipitation_list, arvi_list, sr)
plt.savefig('lagprec_halfmonth.png',dpi=250,bbox_inches = 'tight')

In [ ]:
#precipitation_list = list(meteo_precip["precipitation"])
#temperature_list = list(meteo_temp["temperature"])
shortwave_list = list(grouped_shortwave["shortwaves"]) 
arvi_list = list(indice["ARVI"])

def lag_finder(shortwave_list, arvi_list, sr):
    n = len(shortwave_list)

    corr = signal.correlate(arvi_list, shortwave_list, mode='same') / np.sqrt(signal.correlate(shortwave_list, shortwave_list, mode='same')[int(n/2)] * signal.correlate(arvi_list, arvi_list, mode='same')[int(n/2)])

    delay_arr = np.linspace(-0.5*n/sr, 0.5*n/sr, n)
    delay = delay_arr[np.argmax(corr)]
    print('arvi_list is ' + str(delay) + ' behind shortwave_list')

    plt.figure()
    plt.plot(delay_arr, corr)
    plt.title('Lag: ' + str(np.round(delay, 3)) + 'month')
    plt.xlabel('Lag')
    plt.ylabel('Correlation coefficient')
    plt.show()
   
# Sine sample with some noise and copy to y1 and y2 with a 1-second lag
sr = 60
y = np.linspace(0, 2*np.pi, sr)
y = np.tile(np.sin(y), 5)
y += np.random.normal(0, 5, y.shape)
shortwave_list = y[sr:4*sr]
arvi_list = y[:3*sr]

lag_finder(shortwave_list, arvi_list, sr)
plt.savefig('lagshort_halfmonth.png',dpi=250,bbox_inches = 'tight')

In [ ]:
"""# Partial correlation
ARVI=[]
meteo_precip=[]
meteo_temp=[]
grouped_shortwave=[] """

In [ ]:

"""
shortwave_list = list(grouped_shortwave["shortwaves"]) 
arvi_list = list(indice["ARVI"])

def plot_xcorr(shortwave_list, arvi_list): 
    "Plot cross-correlation (full) between two signals."
    N = max(len(shortwave_list), len(arvi_list)) 
    n = min(len(shortwave_list), len(arvi_list)) 

    if N == len(arvi_list): 
        lags = np.arange(-N + 1, n) 
    else: 
        lags = np.arange(-n + 1, N) 
    c = correlate(shortwave_list / std(shortwave_list), arvi_list / std(arvi_list), 'full') 

    plt.plot(lags, c / n) 
    plt.show() 
    plot_xcorr(shortwave_list, arvi_list)
    plt.savefig('lagshort_halfmonth1.png',dpi=250,bbox_inches = 'tight')

In [ ]:
shortwave_list = list(grouped_shortwave["shortwaves"])
arvi_list =list(indice["ARVI"])

def plot_xcorr(shortwave_l, arvi_l): 
    "Plot cross-correlation (full) between two signals."
    N = max(len(shortwave_l), len(arvi_l)) 
    n = min(len(shortwave_l), len(arvi_l)) 

    if N == len(shortwave_l): 
        lags = np.arange(-N + 1, n) 
    else: 
        lags = np.arange(-n + 1, N) 
    c = correlate(shortwave_l / np.std(shortwave_l), arvi_l / np.std(arvi_l), 'same') 

    plt.plot(lags, c / n) 
    plt.show() 
    plot_xcorr(shortwave_l, arvi_l)
    plt.savefig('lagshort_halfmonth1.png',dpi=250,bbox_inches = 'tight')

    
plot_xcorr(shortwave_list, arvi_list) #Here you excecute it

In [ ]:
len(shortwave_list)

In [ ]:
len(arvi_list)

In [16]:
print(dt.datetime.strptime('8-Jan-18', '%d-%b-%y').date())

2018-01-08
